# Error Handling

In [2]:
// Install dependencies
#r "nuget: xunit, 2.4.2"
#r "nuget: FluentAssertions, 6.11.0"
#r "nuget: CSharpFunctionalExtensions, 2.39.2"


Installed Packages CSharpFunctionalExtensions, 2.39.2 FluentAssertions, 6.11.0 xunit, 2.4.2

In [3]:
using Xunit;
using FluentAssertions;
using FluentAssertions.Extensions;

## Avoid Primitive Obsession

In [5]:
public class VeranstaltungBad
{
    public string Name { get; set; }
    public DateTime Beginn { get; set; }
    public DateTime Ende { get; set; }
}

var date1 = 13.July(2023); // <- see FluentAssertions.Extensions
var date2 = 14.July(2023);

var veranstaltungBad = new VeranstaltungBad
{
    Name = "Bad",
    Beginn = date2,
    Ende = date1
};

//veranstaltungBad.Beginn.Should().BeBefore(veranstaltungBad.Ende);



Error: Xunit.Sdk.XunitException: Expected the date and time to be before <2023-07-13>, but found <2023-07-14>.
   at FluentAssertions.Execution.XUnit2TestFramework.Throw(String message) in /_/Src/FluentAssertions/Execution/XUnit2TestFramework.cs:line 35
   at FluentAssertions.Execution.TestFrameworkProvider.Throw(String message) in /_/Src/FluentAssertions/Execution/TestFrameworkProvider.cs:line 40
   at FluentAssertions.Execution.DefaultAssertionStrategy.HandleFailure(String message) in /_/Src/FluentAssertions/Execution/DefaultAssertionStrategy.cs:line 25
   at FluentAssertions.Execution.AssertionScope.FailWith(Func`1 failReasonFunc) in /_/Src/FluentAssertions/Execution/AssertionScope.cs:line 284
   at FluentAssertions.Execution.AssertionScope.FailWith(Func`1 failReasonFunc) in /_/Src/FluentAssertions/Execution/AssertionScope.cs:line 252
   at FluentAssertions.Execution.AssertionScope.FailWith(String message, Object[] args) in /_/Src/FluentAssertions/Execution/AssertionScope.cs:l

In [6]:
public class VeranstaltungBetter
{
    public string Name { get; set; }
    public DateRange1 Zeitraum { get; set; }
}

public class DateRange1
{
    public DateTime Start { get; } // no public setter
    public DateTime End { get; }   // no public setter

    public DateRange1(DateTime start, DateTime end)
    {
        if (end < start)
        {
            throw new ArgumentException("End date must be after start date.");
        }

        // add more validation here if needed

        Start = start;
        End = end;
    }
}

var range = new DateRange1(date1, date2); // Can never be invalid! -> "Make invalid states unrepresentable"

var veranstaltung = new VeranstaltungBetter
{
    Name = "Bad",
    Zeitraum = range
};

veranstaltung.Zeitraum.Start.Should().BeBefore(veranstaltung.Zeitraum.End);

## Law of Demeter

- ⚠️ avoid "train wrecks" (f. ex. `Person.Address.Street.Nr`)
- 💡 prefer Information Hiding / Encapsulation
- 👉 easier to change in the future

In [14]:
veranstaltung.Zeitraum.Start.Should().BeBefore(veranstaltung.Zeitraum.End);
//            ^^^^^^^^                                       ^^^^^^^^

public class VeranstaltungBetter2
{
    private DateRange1 Zeitraum { get; set; } // private setter

    public string Name { get; set; }
    public DateTime Start => Zeitraum.Start;
    public DateTime End => Zeitraum.End;

    public VeranstaltungBetter2(string name, DateRange1 zeitraum)
    {
        Name = name;
        Zeitraum = zeitraum;
    }
}

var veranstaltung2 = new VeranstaltungBetter2("Bad", range);

veranstaltung2.Start.Should().BeBefore(veranstaltung2.End);

In [8]:
public class Person1
{
    public IEnumerable<Address> Addresses { get; set; }
    public string StreetNumber => string.Join(",", Addresses.Select(x =>x.Street.Nr));
}
public class Address {
    public Street Street { get; set; }
    public string StreetNumber => Street.Nr;
}
public class Street {
    public string Nr { get; set; }
}

var person = new Person1{
    Address = new Address
    {
        Street = new Street
        {
            Nr = "1"
        }
    }
};

person.StreetNumber.Should().Be("1");


## Value Objects

Value Objects (VOs) are one of the building blocks of Domain-Driven Design (DDD).

- equality by structure
- immutable (no life cycle)
- contain as much business logic as possible!

### Equality

- by reference (default in C#)
- by id (often used in combination with persistence)
- by structure

Two VOs are equal when they contain the same values. The don't have to be the same object.

We want the following behaviour:

```csharp
var money1 = new Money(5, "EUR");
var money2 = new Money(5, "EUR");

var areEqual = money1 == money2;

areEqual.Should().BeTrue();
```

To accomplish this, we have to override `GetHashCode` and `Equals`.

Example:

```csharp
override bool Equals(object? obj)
{
    if (obj is not Money other)
        return false;

    return other.Amount == this.Amount &&
        other.Currency == this.Currency;
}

override int GetHashCode() 
{ 
    return HashCode.Combine(Amount, Currency); // Thx, Rider ;-)
}

### Immutability

Once a Value Object has been created, it can never change.

Why is this a good idea?

- a valid VO will always be valid
- simplifies multithreading
- simplifies testing

#### How to ensure Immutability?

- properties should never have public setters
- avoid parameterless constructors (watch out for frameworks!)

### Add logic

We can add more than just validation logic to Value Objects.

Example:

```csharp
class DateRange
{
    // ...

    DateRange ChangeStartDate(DateTime newStartDate)
    {
        // The constructor already contains our validation
        return new DateRange(newStartDate, this.End);
    }
}
```

### Value Objects: Common practices

- nested VOs
- reduce boilerplate code by using a library 
    - f. ex. [CSharpFunctionalExtentions](https://github.com/vkhorikov/CSharpFunctionalExtensions)
- Validate at the boundary of the Core Domain
    - UnvalidatedThing -> ValidatedThing
- change VOs be creating new VOs
- very simple VOs (without any logic) can be replaced by C# `record`s
- Smart Constructor pattern (more on this later...)

#### Nesting Value Objects

Nothing prevents us from nesting Value Objects. Doing so is actually a good practice.

See AddressExample in IDE

#### Validate at the boundary of the Core Domain

We often have unvalidated data at the boundary of our application. This data is often present in the form of Data Transfer Objects (DTOs) or deserialized data (i.e. from JSON).

Example:

```csharp
// Unvalidated
class AddressDTO
{
    string City { get; set; }
    string Street { get; set; }
}
```

Use a dedicated method or a constructor to create a valid `Address` Value Object.

```csharp
// Always valid
class Address
{
    private readonly City _city;
    private readonly Street _street;

    string City => _city.Value;
    string Street => _street.Value;

    Address(AddressDTO dto)
    {
        if (!IsValid(dto))
        {
            throw new InvalidAddressException($"Invalid address: {dto}");
        }

        _city = new City(dto.City);
        _street = new Street(dto.Street);
    }

    bool IsValid(AddressDTO dto)
    {
        // add validation logic here
        return true;
    }

    // ...
}
```

#### C# records vs Value Objects

C# introduced `record`s a while back.

Can't we just use `record`s instead of Value Objects?

Here is Vladimir Khorikov's take on the subject, which I agree with 💯:

[C# 9 Records as DDD Value Objects](https://enterprisecraftsmanship.com/posts/csharp-records-value-objects/)

C# `record`s

- 👍 are immutable by default
- 👍 are very concise
- 👍 equality by value, but difficult to modify, f. ex.
    - ⚠️ collections
    - ⚠️ rounding errors
    - ⚠️ excluding properties from comparison
- 😠 modifying properties using `with` can bypass validation

`record`s are a great match for

- DTOs
- avoiding primitive obsession
- applications not requiring a rich domain model


In [10]:
public record Foo(Guid id)
{ 
    public string Name { get; init; } 
}

var foo = new Foo(Guid.NewGuid()){Name = "Homer"};

public record FirstName(string Value);
public record LastName(string Value);
public record Person(FirstName FirstName, LastName LastName);

## Exceptions

### Is it really an exception?

Consider the following 2 use cases:

- "input data" provided by user is not valid
- database is not reachable

IMHO the first use case should NOT throw an exception...

But this is an oppinionated topic. There are many different approaches to error handling...

Don't use exceptions for control flow. `try/catch` blocks make code harder to read.

Instead, enrich your model to include failure cases.

### Consider custom exceptions

When using exceptions, consider using custom exceptions. This might improve monitoring, tracing, and debugging in production.

Note: This is one of the few occasions where inheritance can be useful.

### Consider custom data types

But, before we continue with custom data types for error handling, we should first take a look at NULL.

## NULL

Have you ever seen this exception?

```csharp
System.NullReferenceException: Object reference not set to an instance of an object.









```

![null](images/null.webp)

Also referred to as "The Billion Dollar Mistake".

### C# 8 nullable reference types

C# 8 introduced nullable reference types.

```csharp
string? name = null;
```

To enable nullable reference types, add the following to your project file:

```xml
<PropertyGroup>
    <Nullable>enable</Nullable>
</PropertyGroup>
```

Many problems around NULL can be solved by using nullable reference types. 

Also, Value Objects and records are always non-nullable.

In [15]:
public int Intify(string s)
{
    if (int.TryParse(s, out var result))
    {
        return result;
    }

    return -1; // 😠
}

var result1 = Intify("123");
result1.Should().Be(123);

var result2 = Intify("abc");
result2.Should().Be(-1);

In [16]:
public class Data
{
    public string Name;
}

public class Do
{
    public Data CreateData() => null; // ⚠️

    public string CreateAndUseData()
    {
        var data = CreateData();
        // kein null-Check -> ist dem Compiler egal
        return data.Name; // 💥
    }
}

var doer = new Do();
Action act = () => doer.CreateAndUseData();
act.Should().Throw<NullReferenceException>();

## Option

...slowly entering the world of functional programming...

Take the following pseudo code:

```txt
type Option<T> = Some(T) | None
```

- `Some` contains a value
- `None` contains no value

How would we use a type like this?

```csharp
public Option<int> IntifyOption(string input)
{
    return int.TryParse(input, out var result)
        ? Option<int>.Some(result)
        : Option<int>.None;
}
```

How do we access the wrapped value? 👉 `Match`

```csharp
public string Stringify<T>(Option<T> option)
{
    return option.Match(
        some: value => value.ToString(),
        none: () => "No value present");
}
```

Advantages:

- 👍 very explicit method signature (return type is `Option<int>`)
- 👍 `Match` requires us to handle both cases!
- ⚠️ Warning: C# still allows us to assign `null` to `Option<T>`...

## C# libraries

- [LaYumba.Functional](https://github.com/la-yumba/functional-csharp-code): Great for learning functional programming in C#
- [CSharpFunctionalExtentions](https://github.com/vkhorikov/CSharpFunctionalExtensions): Great for production code, simpler than LanguageExt
- [LanguageExt](https://github.com/louthy/language-ext): Great for production code, has many features, but has a steep learning curve


## A new "wrapper": Result

`Option<T>` has 2 states: `Some(T)` and `None`. 

Wouldn't it be nice to know why `None` is `None`?

👉 That is what `Result<T, TError>` is for.

⚠️ Switch to IDE ⚠️